In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
import scipy.linalg

In [2]:
def rot2D(theta):
    return np.array([math.cos(theta), -math.sin(theta), math.sin(theta), math.cos(theta)]).reshape(2,2)

def rot3D(theta):
    o = np.eye(3)
    o[:2,:2] = rot2D(theta)
    return o

def numeraical_jacobian(pos1, pos2, model):
    eps = 1e-12
    pos_H = []
    neg_H = []
    for i,x in enumerate(pos1):
        v = copy.copy(pos1)
        v[i] = x + eps
        pos_H.append(model(v, pos2))
    for i,x in enumerate(pos2):
        v = copy.copy(pos2)
        v[i] = x + eps
        pos_H.append(model(pos1, v))

    for i,x in enumerate(pos1):
        v = copy.copy(pos1)
        v[i] = x - eps
        neg_H.append(model(v, pos2))
    for i,x in enumerate(pos2):
        v = copy.copy(pos2)
        v[i] = x - eps
        neg_H.append(model(pos1, v))
        
    H = (np.vstack(pos_H).T - np.vstack(neg_H).T) / (2*eps)
    return H


def odom_model(pos1, pos2):
#     x1, y1, t1 = pos1
#     x2, y2, t2 = pos2
#     H = np.eye(3)
#     R = rot2D(t2-t1)
#     odom = pos2[:2] - np.dot(R, pos1[:2].reshape(-1,1)).reshape(-1,)
    return pos2-pos1

def odom_jacobian(pos1, pos2):
    x1, y1, t1 = pos1
    x2, y2, t2 = pos2
#     H = np.array([[-math.cos(t2-t1), math.sin(t2-t1), -x1*math.sin(t2-t1) - y1*math.cos(t2-t1),
#                   1, 0, +x1*math.sin(t2-t1) + y1*math.cos(t2-t1)],
#                  [-math.sin(t2-t1), -math.cos(t2-t1), x1*math.cos(t2-t1) - y1*math.sin(t2-t1),
#                   0, 1, -x1*math.cos(t2-t1) + y1*math.sin(t2-t1)],
#                  [0, 0, -1, 0, 0, 1]])
    H = np.zeros((3,6))
    H[0,0] = -1
    H[0,3] = 1
    H[1,1] = -1
    H[1,4] = 1
    H[2,2] = -1
    H[2,5] = 1
    return H

def measurement_model_w(pose_w, plane_w):
    n_w = plane_w[:3].reshape(-1,1)
    d_w = plane_w[-1]
    x,y,theta = pose_w
    h = np.array([x,y,1]).reshape(-1,1)
    n_l = np.dot(rot3D(theta), n_w).reshape(-1,)
    d_l = (np.dot(n_w.T, h) + d_w).reshape(-1,) / np.linalg.norm(n_w)
    return np.hstack([n_l,d_l])

# TODO: pass in normalized n
def invert_measurement_l(pose_w, plane_l):
    n_l = plane_l[:3].reshape(-1,1)
    d_l = plane_l[-1]
    x,y,theta = pose_w
    h = np.array([x,y,1]).reshape(-1,1)
    n_w = np.dot(rot3D(theta).T, n_l).reshape(-1,)
    
    d_w = (-np.dot(n_w.T, h) + d_l).reshape(-1,) 
    return np.hstack([n_w,d_w])

def measurement_jacobian(pose, plane):
    x,y,theta = pose
    nx_w,ny_w,nz_w,d_w = plane
    
    H = np.array([[0, 0, -math.sin(theta)*nx_w - math.cos(theta)*ny_w, math.cos(theta), -math.sin(theta), 0, 0],
                  [0 ,0, math.cos(theta)*nx_w - math.sin(theta)*ny_w, math.sin(theta), math.cos(theta), 0, 0],
                  [0, 0, 0, 0, 0, 1, 0],
                  [nx_w, ny_w, 0, x, y, 1, 1]]) # re-compute normalized jacobian
    
    return H

In [3]:
x0 = np.array([0,0,0])
x1 = np.array([2,3,np.deg2rad(45)])
x2 = np.array([5,5,np.deg2rad(-45)])

plane1 = np.array([0.707, 0.707, 0, 5])
plane2 = np.array([-0.707, 0.707, 0, 2])

gt = np.hstack([x0, x1, plane1, plane2])

def odom_noise():
    return np.hstack([np.random.normal(0, 0.5, 2), np.random.normal(0, 0.1, 1)])

def measurement_noise():
    return 0
#     return np.random.normal(0,1, 4)

In [4]:
odom1 = odom_model(x0,x1) + odom_noise()
odom2 = odom_model(x1,x2) + odom_noise()

measurement11 = measurement_model_w(x0, plane1) + measurement_noise()
measurement12 = measurement_model_w(x0, plane2) + measurement_noise()
measurement21 = measurement_model_w(x1, plane1) + measurement_noise()
measurement22 = measurement_model_w(x1, plane2) + measurement_noise()
landmark_measurements = np.hstack([measurement11,measurement12,measurement21,measurement22])

In [18]:
std_x = np.array([0.1, 0.1, 0.05]) # x, y, theta
std_l = np.array([0.1, 0.1, 0.1, 0.1]) # nx, ny, nz, d

In [6]:
s_x0 = x0
s_l = np.hstack([invert_measurement_l(x0, measurement11), invert_measurement_l(x0,measurement12)])
print(s_l)
s_x1 = np.hstack([x0, x0+odom1])

s = np.hstack([s_x1, s_l])
print(s)
def generateAB(s_x1, s_l, odom1, landmark_measurements, std_x, std_l):
    A_x1 = np.zeros((6,14))
    A_x1[:3,:3] = np.eye(3)
    odom_jac = odom_jacobian(s_x1[0:3], s_x1[3:6])
    A_x1[3:,:6] = odom_jac

    A_l1 = np.zeros((16,14))
    #measurement_jac1 = measurement_jacobian(s_x1[:3], s_l[:4])
    measurement_jac1 = numeraical_jacobian(s_x1[:3], s_l[:4], measurement_model_w)
    A_l1[:4,0:3] = measurement_jac1[:, 0:3]
    A_l1[:4,6:10] = measurement_jac1[:, 3:7]
    #measurement_jac2 = measurement_jacobian(s_x1[:3], s_l[4:])
    measurement_jac2 = numeraical_jacobian(s_x1[:3], s_l[4:], measurement_model_w)
    A_l1[4:8,0:3] = measurement_jac2[:, 0:3]
    A_l1[4:8,10:14] = measurement_jac2[:, 3:7]

    #measurement_jac1 = measurement_jacobian(s_x1[3:], s_l[:4])
    measurement_jac1 = numeraical_jacobian(s_x1[3:], s_l[:4], measurement_model_w)
    A_l1[8:12,3:6] = measurement_jac1[:, 0:3]
    A_l1[8:12,6:10] = measurement_jac1[:, 3:7]
    #measurement_jac2 = measurement_jacobian(s_x1[3:], s_l[4:])
    measurement_jac2 = numeraical_jacobian(s_x1[3:], s_l[4:], measurement_model_w)
    A_l1[12:16,3:6] = measurement_jac2[:, 0:3]
    A_l1[12:16,10:14] = measurement_jac2[:, 3:7]

    m_x1 = np.hstack([np.array([0,0,0]), odom1])
    p_x1 = np.hstack([np.array([0,0,0]), odom_model(s_x1[:3], s_x1[3:])])
    b_x1 = m_x1 - p_x1

    m_l1 = landmark_measurements
    p_l1 = np.hstack([measurement_model_w(s_x1[:3], s_l[:4]), measurement_model_w(s_x1[:3], s_l[4:8]),
                      measurement_model_w(s_x1[3:6], s_l[:4]), measurement_model_w(s_x1[3:6], s_l[4:8])])
    b_l1 = m_l1 - p_l1

    A = np.vstack([A_x1, A_l1])
    b = np.hstack([b_x1, b_l1])
    return A,b

A,b = generateAB(s_x1, s_l, odom1, landmark_measurements)
print(A.shape, b.shape, s.shape)

[ 0.707       0.707       0.          5.00075517 -0.707       0.707
  0.          2.00030207]
[ 0.          0.          0.          1.81957119  2.60367803  0.68605272
  0.707       0.707       0.          5.00075517 -0.707       0.707
  0.          2.00030207]
(22, 14) (22,) (14,)


In [7]:
def gauss_newton(s, odom1, landmark_measurements, max_iter=1000):
    for i in range(max_iter):
        s_x1 = s[:6]
        s_l = s[6:]
        A,b = generateAB(s_x1, s_l, odom1, landmark_measurements)
        dx = scipy.linalg.solve(np.dot(A.T,A),np.dot(A.T,b))
        s_new = s + dx
        _,b_new = generateAB(s_new[:6], s_new[6:], odom1, landmark_measurements)
        if(np.linalg.norm(b_new) > np.linalg.norm(b)):
            print("Error going up - breaking", i)
            break
        s = s_new
        if(np.linalg.norm(b_new - b) < 1e-12):
            print("Converged",i)
            break
    return s

In [16]:
out = gauss_newton(s, odom1, landmark_measurements)
print(s)
print(np.round(out,3))
print(gt)

Error going up - breaking 5
[ 0.          0.          0.          1.81957119  2.60367803  0.68605272
  0.707       0.707       0.          5.00075517 -0.707       0.707
  0.          2.00030207]
[ 0.     0.     0.022  1.896  2.712  0.758  0.683  0.729  0.     5.125
 -0.686  0.726  0.     2.017]
[ 0.          0.          0.          2.          3.          0.78539816
  0.707       0.707       0.          5.         -0.707       0.707
  0.          2.        ]


In [13]:
s_error = np.linalg.norm(gt-s)
o_error = np.linalg.norm(gt-out)
print(s_error, o_error)

0.4466495727304946 0.33571930697534225


In [14]:
print(np.round(abs(gt[:6]-out[:6]),5))

[1.0000e-04 1.2000e-04 2.2160e-02 1.0447e-01 2.8751e-01 2.7460e-02]


In [11]:
print(np.round(abs(gt[:6]-s[:6]),5))

[0.      0.      0.      0.18043 0.39632 0.09935]


# TODO:
- add standard deviations
- recompute jacobian
- Test: expect that if we trust landmarks a lot 